>Conda Environment: `blockchain39` (iMac)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Ul1OjDmr_WF_F2mES17AYiMlG2Un3QXn?usp=sharing)

In [ ]:
# MOUNT GOOGLE DRIVE
# ... in case you want to store data
from google.colab import drive
drive.mount('/content/drive')

---
#  Week 1: HELLO BLOCKCHAIN WORLD
---

#### Setup & Import relevant Libraries for this Course

In [1]:
%%capture [--no-stdout]

# SETUP LIBRARIERS (ONLY ONCE)
%pip install bl101

In [1]:
# IMPORT LIBRARIES

# Standard library imports
import sys
import os
import json
import pkg_resources
import time as time
import requests

# Third party imports
import solcx
from web3 import Web3 
from web3.auto.gethdev import w3
from web3 import EthereumTesterProvider
import hashlib
from Crypto.Hash import keccak
from dotenv import load_dotenv

# UnpackAI Trainings Module
import bl101

# Constants
ENVIRONMENT = pkg_resources.resource_filename('bl101', '.env')
CONTRACT_DIR = pkg_resources.resource_filename("bl101", "contracts")

# Load environment variables from .env in bl101
load_dotenv(ENVIRONMENT)

True

# Hello World

## General Workflow
Let's jump directly into the blockchain ecosystem by creating a **Hello World** example, i.e. we will write a program that stores *Hello World* to the blockchain.   
>1️⃣ create `.sol` --> 2️⃣ read & compile `.sol` --> 3️⃣ build contract --> 4️⃣  deploy --> 5️⃣ interact with contract on blockchain

## 1️⃣. Create `helloworld.sol`

We skip this step by now and use the `helloworld.sol` contract we already written. Later we will learn more about creating so called *smart contracts*. Creating the .sol file is the coding part in the narrower sense of working. Solidity is coding language which is close to *javascript*. We later will use online tools which help us to create smart contracts.

## 2️⃣. Read and compile `helloworld.sol`

For the beginning we use the `helloworld.sol` contract which we provide in our `bl101` module:

In [2]:
# Load Helloworld contract from bl101 package
contract_file = os.path.join(CONTRACT_DIR, 'helloworld.sol')
with open(contract_file, "r") as file:
    contract = file.read()

In [3]:
print(contract)

pragma solidity ^0.8.12;

contract HelloWorld {

string s = "Hello World from Shanghai";

  function read_message() public view returns (string memory) {
      return s;
      }

function write_message(string calldata t) public {
        s = t;
    }

}


👆🏻 Our contract has only a few lines of code consisting mainly of two functions for reading and writing this variable.

In [4]:
solcx.install_solc('0.8.12') # install the right solidity version

compiled_contract = solcx.compile_standard(
    {
    "language": "Solidity",
    "sources": {"helloworld.sol": {"content": contract}},
    "settings": {"outputSelection": {"*": {"*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] }}},
    
    },
    solc_version = "0.8.12"
)
# -> creates dict with keys: errors, sources & >>>contracts<<<

Compiling results in a `dictionary`:

In [5]:
type(compiled_contract)

dict

We can print out the whole `compiled_contract`:

In [6]:
compiled_contract

{'contracts': {'helloworld.sol': {'HelloWorld': {'abi': [{'inputs': [],
      'name': 'read_message',
      'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'string', 'name': 't', 'type': 'string'}],
      'name': 'write_message',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'}],
    'evm': {'bytecode': {'functionDebugData': {'extract_byte_array_length': {'entryPoint': 308,
        'id': None,
        'parameterSlots': 1,
        'returnSlots': 1},
       'panic_error_0x22': {'entryPoint': 261,
        'id': None,
        'parameterSlots': 0,
        'returnSlots': 0}},
      'generatedSources': [{'ast': {'nodeType': 'YulBlock',
         'src': '0:516:1',
         'statements': [{'body': {'nodeType': 'YulBlock',
            'src': '35:152:1',
            'statements': [{'expression': {'arguments': [{'kind': 'number',
                 

But really important are only two parts: 

In [7]:
# ABI
abi = compiled_contract["contracts"]['helloworld.sol']["HelloWorld"]["abi"]
# BYTECODE
bytecode = compiled_contract["contracts"]['helloworld.sol']["HelloWorld"]["evm"]["bytecode"]["object"]

The `ABI` contains all the metadata of that contract and the `bytecode` the actual code in bytecode format:

In [8]:
abi

[{'inputs': [],
  'name': 'read_message',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'string', 'name': 't', 'type': 'string'}],
  'name': 'write_message',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [9]:
print(len(bytecode))
bytecode[:500]

2694


'60806040526040518060400160405280601981526020017f48656c6c6f20576f726c642066726f6d205368616e67686169000000000000008152506000908051906020019061004f929190610062565b5034801561005c57600080fd5b50610166565b82805461006e90610134565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b80821115610101576000816000905550'

## 3️⃣. Build Contract 📜

The next step is to connect to the `Ethereum` blockchain with the help of the widely used python library `web3.py`:   

In [10]:

# to interact with ethereum blockchain we need ...
# we provide a few keys in our package for the mainnet and most common testnets. Anyhow, we highly encourage you to go to ... and get your own key.
# 🚩 To-Do: explain what Infura is and why and how to get key

infura_url_rinkeby = 'https://rinkeby.infura.io/v3/' + os.environ.get("INFURA")  #my Infura project ID/uri for Rinkeby (stored in bl101)

w3 = Web3(Web3.HTTPProvider(infura_url_rinkeby))
w3.isConnected()

True

👆🏻 For this we also use an API-Servie called `Infura` (*more on this later ...*)

In [11]:
# build web3.py contract object
helloworld = w3.eth.contract(abi=abi, bytecode=bytecode)

## 4️⃣. Deploying 🚀 

In [12]:
wallet_address = os.environ.get("WALLETADDRESS") # <-from bl101, my personal Metamask account (? shall we provide one unpackai metamask account for the students?)
nonce = w3.eth.get_transaction_count(wallet_address)

nonce

121

🚩 ToDo: 
- explain `nonce`
- decide wether to provide `MetaMask` credentials or force students to open own MetaMask account

In [13]:
#ropsten_chain = 3
rinkeby_chain = 4
transaction = helloworld.constructor().buildTransaction(
    {
        "chainId": rinkeby_chain, 
        "from": wallet_address, 
        "nonce": nonce, 
        "gasPrice": w3.eth.gas_price})

In [14]:
transaction

{'value': 0,
 'gas': 290512,
 'chainId': 4,
 'from': '0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0',
 'nonce': 121,
 'gasPrice': 1000003199,
 'data': '0x60806040526040518060400160405280601981526020017f48656c6c6f20576f726c642066726f6d205368616e67686169000000000000008152506000908051906020019061004f929190610062565b5034801561005c57600080fd5b50610166565b82805461006e90610134565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b808211156101015760008160009055506001016100e9565b5090565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b6000600282049050600182168061014c57607f821691505b602082108114156101605761015f610105565b5b50919050565b6103ce806101756000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c806380b1f30c1461003b578063eb206af414610

Now I need to sign ✍️ this transaction, i.e.: I need to proof that I am the owner of that wallet. I do so with my `private key` from my wallet.

In [15]:
# !! Metamask Private Key - again: should we open a meta mask account for the course?
private_key = "0x" + "YOURMETAMASKPRIVATEKEY"

signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=private_key)

In [16]:
signed_transaction

SignedTransaction(rawTransaction=HexBytes('0xf9059579843b9ad67f83046ed08080b9054360806040526040518060400160405280601981526020017f48656c6c6f20576f726c642066726f6d205368616e67686169000000000000008152506000908051906020019061004f929190610062565b5034801561005c57600080fd5b50610166565b82805461006e90610134565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b808211156101015760008160009055506001016100e9565b5090565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b6000600282049050600182168061014c57607f821691505b602082108114156101605761015f610105565b5b50919050565b6103ce806101756000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c806380b1f30c1461003b578063eb206af414610057575b600080fd5b6100556004803603810190610050919061022f565b610075565b0

In [17]:
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

In [18]:
transaction_hash

HexBytes('0xe88a294889bed75fe51bd285178710a6e09c31467d518d6f8fd90f048380d90a')

And this finally sends the transaction to the blockchain:

In [19]:
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

We have deployed our `helloworld` program. *Our smart contract now lives on the blockchain.* 🥳

## 5️⃣. Interacting with our contract on the blockchain 👩‍❤️‍👨  

In [21]:
contract_instance = w3.eth.contract(address = transaction_receipt.contractAddress,
                                    abi = abi)

In [22]:
contract_instance.functions.read_message().call()

'Hello World from Shanghai'

*Cool!* We are able to read our contract.

But what if we want to change our *HelloWorld* Text?   
Remember, we have that `write_message` function in our contract to change the greeting text. For this we need to change the state of our contract on the blockchain.  
So instead of `Hello from Shanghai!`, we want to store `Hello from Hamburg!` in our contract on the blockchain.   

To do so, we need to send the new message to our `write_message` function by updating our contract:

In [23]:
nonce = w3.eth.get_transaction_count(wallet_address)

# change message by calling the write_message() function from our helloworld contract 
update_helloworld_transaction = contract_instance.functions.write_message("Hello from Hamburg!").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": rinkeby_chain,
        "from": wallet_address,
        "nonce": nonce
    }
)

In [24]:
update_helloworld_transaction

{'value': 0,
 'gas': 29840,
 'gasPrice': 1000003197,
 'chainId': 4,
 'from': '0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0',
 'nonce': 122,
 'to': '0x4410b822276E8107D4bE5A2709741767ea43399d',
 'data': '0x80b1f30c0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000001348656c6c6f2066726f6d2048616d627572672100000000000000000000000000'}

The next steps are the same as we did when we deployed the contract. Again, we need to:
1. sign transaction
2. build hash
3. get receipt

In [25]:
# this might take a while (depending on how big the contract is)
t0=time.time()

# 1.
signed_transaction = w3.eth.account.sign_transaction(update_helloworld_transaction,
                                                        private_key=private_key)
# 2.
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# 3.
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

t1=time.time()
print(f"Execution time: {t1-t0:.2f}s")

Execution time: 12.04s


In [26]:
contract_instance.functions.read_message().call()

'Hello from Hamburg!'

*Voilà!* Our new greeting message! Freshly arrived from `Ethereum` blockchain 🕺💃🥳

# Mainnet vs. Testnet

### What Is a Testnet?
A testnet is a test blockchain network that works to run and test blockchains or blockchain projects before they are ready to be launched. It is a simple way for programmers and developers to create, modify, and test the functionalities of their project, as well as monitor its performance before making it accessible to the public. Here, developers can troubleshoot any issues and fix any bugs.

In a testnet environment, multiple tests can be run repeatedly, which allows for a performance comparison and means that consistency can be checked. By running independently to the mainnet, testnets allow for the full testing of a blockchain project without interfering with the transactions on the mainnet. This sandbox model allows developers to take risks, experiment, and thus create the best possible model to launch.

As a prototype, a testnet should never be used to transfer anything of value, and thus testnets use fake money, or tokens without value, to run their protocols. Developers will often also use a testnet to build and trial their own blockchain, and once satisfied will then go and launch them on their own. Testnets allow for a faster and safer launching of the mainnet.

### What Is a Mainnet?
A mainnet is the “main” “net,” or network, that a blockchain or blockchain project is run on. This is the stage that logically comes after the completion of all the necessary trials on the testnet. A mainnet consists of a fully launched net where cryptocurrency transactions can be processed efficiently, verified accurately, and recorded securely. By deploying a mainnet, its developers are stating that they have full trust in their blockchain’s capacity. As a fully functioning blockchain, mainnets can be used to send and receive any transaction, in the form of cryptocurrency or non-fungible tokens (NFTs), among others, or transfer information. Additionally, a mainnet can run a project that requires that specific blockchain’s protocol, for example DApps on the Ethereum mainnet. <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)


![](https://github.com/dirkkalmbach/bl101/blob/main/bl101/images/mainnet-and-testnet-differences.jpg?raw=true)
<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)

## Connect to Ethereum Test Node

## Connect to Ethereum with HTTP Provider

1. Got to [infura](https://infura.io) and open a free account
2. Create a new `Ethereum` project 
3. Chose `Mainnet` endpoint and copy the endpoint url. You can also copy the url of some testnets, like `Rinkeby` or `Kovan`. 

![](https://github.com/dirkkalmbach/bl101/blob/main/bl101/images/Infura.png?raw=true)

In [27]:
# Should we open an infura account for unpackAI? So for the beginning students can use it right away without registering there own ...
# these are my infura accounts apis (for now):


infura_url_mainnet = 'https://mainnet.infura.io/v3/' + os.environ.get("INFURA")
infura_url_testnet_rinkeby = 'https://rinkeby.infura.io/v3/' + os.environ.get("INFURA")
infura_url_testnet_kovan = 'https://kovan.infura.io/v3/' + os.environ.get("INFURA")

provider = infura_url_mainnet
w3 = Web3(Web3.HTTPProvider(provider))
w3.isConnected()

True

We are now connected with the `mainnet` - the *real* ethereum blockchain.

## How to interact with Ethereum Blockchain from Python?

The following code shows us the latest block which was added to the blockchain. It contains all the information we need to understand how blockchain works:  

In [28]:
latest_block = w3.eth.get_block('latest')
latest_block

AttributeDict({'baseFeePerGas': 88445354286,
 'difficulty': 14727707643811661,
 'extraData': HexBytes('0x75732d77657374312d32'),
 'gasLimit': 30029295,
 'gasUsed': 30010459,
 'hash': HexBytes('0x6f047101b325d67429298fafdc578979853f3d5db1e343f420db33d2794084d2'),
 'logsBloom': HexBytes('0xfab373f36fa2d1f7f8fa3be9febfdbe39dc7ffff9dfc76fd70f92dffdf7f0af3be6dd759d0f9d161755affd75cbbdd673f6baba73f1fbfe71717fedb4fbcfff5be5cc394dddfaf7fddbe7f2f79fdf7e6767f73875373d43fbd73decbfaf8df39dfefb1dd4e6f73b37fe79ef88b356dd3ff8ddff7f83ffff67b1fdd36caf9a97f7da7d5ecfe7c59efdefe20778b8f09775e76decdf7dc8c7d7f26f0db59bb7fffb3cfe8ffbbcf7edb7f93769cf95fbcbe4bf7533f377eb8ba432b438f1a7e6ffd4f5df7e3552c75f32b5ddd9bdbcd3cc474f8fbd6ffff9ed57f95fdd7ee3e296b1ebb7d4fbad5fb5ed3eed7290afc7fe7bfef74fcf7f21d7fbd3dfbfc9b57dbe7'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0x52656adf594767df3e28f321d26b1670e1e8ad67348e9bc8b2ae4503a2ea7e27'),
 'nonce': HexBytes('0x4170e670f9019cb2'),
 'num

👆🏻 the output contains a lot of useful information which are essential to understand the blockchain ecosystem:

|Attribute|Description|
|---|---|
|`gas_limit`||
|`gasUsed`|the *price* to execute this transaction, e.g. how many wei the sender had to pay |
|`hash`|transaction address|
|`miner`|the address of the miner who *solved* the hashing algorithm|
|`difficulty`| ? |
|`logsBloo`| a list of all ... ???|
|...|...|

#### Check if an address is a valid ethereum address

One interesting information from `latest_block` is the address of the miner who finally succeeded in solving the hashing problem and earned the *gas* for this:

In [29]:
miner_addr = latest_block.miner
miner_addr

'0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8'

We can always check if an address is a valid ethereum address by using w3's `isAddress()` function: 

In [30]:
w3.isAddress(miner_addr) # <- the value from miner ⬆️

True

#### Hashing Algorithm

So, why do these addresses have this strange (cryptic 😉) format? Why not instead *Miner-Joes-Eth-Account* for example? Well, actually it is possible to chose a name.You can register a meaningful name for your ethereum account - pretty much like websites have ip addresses (e.g. `23.202.231.167`) and corresponding domain names (e.g. [unpack.ai](www.unpack.ai)). *But we will come to this later ...*  

Blockchain addresses are also called *hashes*, i.e. because they are the result of a so-called hashing-function. Hashing functions convert an input value (text/numbers) into an output value with a predefined format in a way that the same input value will always produce the same output, but it's impossible to find the input value by only knowing the output value. A common usecase is password hashing. The following code shows a widely used hashing function which is often used for password encryption:

In [31]:
password = b"I-love-unpackai"
hashlib.sha256(password).hexdigest()

'202de8fca7e26837495f39d3b2ade505d71f17c34f5f66b58fb7595d6e1648d8'

👆🏻 you can run the cell again, and it will produce always the same hash. But it's not possible (or very hard) extract the password by only knowing the hash. 

Passwords are usually stored as such a hashes so that nobody beside the user knows the actual password. Whenever the user logsin the system builds the hash like above and compares it with the stored hash value in the database. If both values are right, the system grants access.

The hashing algorithm in the ethereum blockchain is `Keccak-256`. "Keccak is nice that it has arbitrary inputs and an infinite input space. This enables one to 'make a hash' of a super large file where each input causes the internal state to scramble up some more. The hash should entirely change if a single bit of data in the source is different. (...)  It means your password could be a million chars long maybe. It's stored on disk as a hash, much smaller in size."<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)

In [32]:
k = keccak.new(digest_bits=256)
k.update(b'age') # python3: input must be in bytes
print(k.hexdigest())

36383cc9cfbf1dc87c78c2529ae2fcd4e3fc4e575e154b357ae3a8b2739113cf


*.... but let's go back to our Miner:*  

We can even see how much **Ether** this miner has on his/her account:

In [33]:
balance = w3.eth.get_balance(miner_addr)
print("Wei: ", balance)

Wei:  2960875983239354219295


The output is in `Wei`. Wei is the subunit of the `Ether` currency - like *cent* to *dollar* or *euro*. So we often want to convert Wei to Ether:

In [34]:
balance_eth =  w3.fromWei(balance, 'ether')
print("Ether: ", balance_eth)

Ether:  2960.875983239354219295


... or in $ or €:

In [ ]:
# I put the get_crpyto_data() function out of bl101 for now as it took too long to install (probably because of plotly 🧐 -> need to refactor it!)

#data_eth = get_crypto_data(tickers='ETH-USD')
#print("US-$: ", round(float(data_eth.tail(1)["Close"]) * float(balance_eth) , 2))

In [35]:
print("US-$: ", round(2000 * float(balance_eth) , 2)) # with an estimated 1:2000 exchange rate

US-$:  5921751.97


> 😲 Wow! Thats quite a lot of money! Mining must be a very profitable business.

### Ehterium Name Service
We mentioned that *Ethereum addresses* can have names. The Service behind it is called `Ethereum Names Service` (ENS). Every ethereum name ends with **.eth** and like web domains you are registering for a specific time period. ENS provides custom names for ethereum addresses like domain names for websites. You can purchase an ENS [here](https://manager.ens.domains). The current price (as of May 2022) for register an Ethereum address is 0.002 ETH (registration fee) + gas. The estimated gas depends on the name. E.g. for registering **unpackai.eth** the costs would be about US$23 for one year: 

<img src='https://github.com/dirkkalmbach/bl101/blob/main/bl101/images/ENS_for_unpackai.png?raw=true' width=500px>

Let's see if our miner has an **Ethereum Name**:

In [36]:
print(w3.ens.name(miner_addr))

None


Hmm, 🤔. Apparently, not. 

*But lets check another address ...*

In [37]:
# get name of ethernet account
print(w3.ens.name("0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045"))

#get address of account holder
print(w3.ens.address('vitalik.eth'))

vitalik.eth
0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045


# How much crypto in my wallet? (connect to `Metamask`)

To know how much Ether is in your wallet is of course important. Unfortuntely, we cannot use the `get_balance` function to access our test account wallets from **Metamask**:

In [46]:
# How much balance in my Metamask wallet on Rinkeby Testnet?
# 1. Connect to Testnet
provider = infura_url_testnet_rinkeby
w3 = Web3(Web3.HTTPProvider(provider))

wallet_address = os.environ.get("WALLETADDRESS") # <- Metamask account
wei = w3.eth.get_balance(wallet_address)
eth =  w3.fromWei(int(wei), 'ether')
print(f"{round(eth,2)} Ether in wallet {wallet_address}")

3.28 Ether in wallet 0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0


We can also see our balance directly on the Ethereum blockchain by adding our wallet address to the rinkebey testnet url, like 
https://**TESTNET**.etherscan.io/address/**WALLETADDRESS**


 <img src='https://github.com/dirkkalmbach/bl101/blob/main/bl101/images/testnet_account_on_etherscan.png?raw=true' width=500px>

In the same way, I can also check how much *real* ether I have in my Metamask wallet. For this we have to switch to the **mainnet**:

In [40]:
# How much balance in my Metamask wallet on Rinkeby Testnet?
# 1. Connect to Testnet
provider = infura_url_mainnet
w3 = Web3(Web3.HTTPProvider(provider))

wallet_address = os.environ.get("WALLETADDRESS") # <- Metamask account
w3.eth.get_balance(wallet_address)

0

Apparantly, there is no money in my wallet 🥺. But feel free to send me some Ether 🤑🤗.   
*(In later courses we will learn how to do this with `Python`.)*

## Get Balance via Ethernet API
- this is just an additional way to get your balance by scraping ethereum website with API key from ethereum.

In [43]:
ApiKeyToken = os.environ.get("ETHERSCAN_API_TOKEN") # <- Etherscan API KEY (stored in bl101)

url =f"https://api-rinkeby.etherscan.io/api?module=account&action=balance&address={wallet_address}&tag=latest&apikey={ApiKeyToken}"

response = requests.get(url,
        headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, '
                               'like Gecko) Chrome/50.0.2661.102 Safari/537.36'})
                               

In [45]:
wei = response.json()["result"]

eth =  w3.fromWei(int(wei), 'ether')
print(f"{round(eth,2)} Ether in wallet {wallet_address}")

3.28 Ether in wallet 0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0


---

# 📚 Reference

<a name="cite_note-1"></a>1. [^](#cite_ref-1) What are Mainnet and Testnet? [🌏 phemex.com](https://phemex.com/academy/what-are-mainnet-and-testnet)

<a name="cite_note-2"></a>2. [^](#cite_ref-2) Crypto Mainnet vs Testnet: what is the difference? [🌏 masterthecrypto.com](https://masterthecrypto.com/mainnet-vs-testnet-whats-the-difference/)

<a name="cite_note-2"></a>3. [^](#cite_ref-3) How does the Keccak256 hash function work? [🌏 ethereum.stackexchange.com](https://ethereum.stackexchange.com/a/45585)